In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('../.aws/credentials')  # replace with your filename/path


In [ ]:
aws_access_key_id = config['default']['aws_access_key_id']
aws_secret_access_key = config['default']['aws_secret_access_key']

In [ ]:
import boto3
from pathlib import Path

# ---- Config ----
bucket = "housing-regression-data-eu"   # replace with your bucket name
region = "eu-central-1"                 # your AWS region

# Set project root as parent of the notebooks folder
PROJECT_ROOT = Path.cwd().parent
#PROJECT_ROOT = ".."
local_data_dir = PROJECT_ROOT / "data" / "processed"
local_model_dir = PROJECT_ROOT / "models"

session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

s3 = session.client("s3", region_name=region)

In [ ]:
# ---- Helper function ----
def upload_file(local_path: Path, s3_key: str):
    if not local_path.exists():
        print(f"❌ File not found: {local_path}")
        return
    print(f"⬆️ Uploading {local_path} → s3://{bucket}/{s3_key}")
    s3.upload_file(str(local_path), bucket, s3_key)

# ---- Upload required datasets ----
upload_file(local_data_dir / "feature_engineered_holdout.csv", "processed/feature_engineered_holdout.csv")
upload_file(local_data_dir / "cleaning_holdout.csv", "processed/cleaning_holdout.csv")
upload_file(local_data_dir / "feature_engineered_train.csv", "processed/feature_engineered_train.csv")

# ---- Upload model ----
upload_file(local_model_dir / "xgb_best_model.pkl", "models/xgb_best_model.pkl")